In [369]:
import pandas as pd
import requests

Districts, town, postal table

In [370]:
# read districts excel 
# https://www.ura.gov.sg/realEstateIIWeb/resources/misc/list_of_postal_districts.htm
districts = pd.read_excel('../data/districts.xlsx')

In [371]:
districts.head()

,Postal District,Postal Sector,General Location
0,1,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People’s Park"
1,2,"07, 08","Anson, Tanjong Pagar"
2,3,"14, 15, 16","Queenstown, Tiong Bahru"
3,4,"09, 10","Telok Blangah, Harbourfront"
4,5,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New..."


In [372]:
districts.columns

Index(['Postal District', 'Postal Sector', 'General Location'], dtype='object')

In [373]:
# read districts df and convert Postal Sector column which is a string of items separated by comma, to a list
districts['Postal Sector'] = districts['Postal Sector'].str.split(',')
# same for General Loation
districts['General Location'] = districts['General Location'].str.split(',')


In [374]:
districts.head()

,Postal District,Postal Sector,General Location
0,1,"[01, 02, 03, 04, 05, 06]","[Raffles Place, Cecil, Marina, People’s Park]"
1,2,"[07, 08]","[Anson, Tanjong Pagar]"
2,3,"[14, 15, 16]","[Queenstown, Tiong Bahru]"
3,4,"[09, 10]","[Telok Blangah, Harbourfront]"
4,5,"[11, 12, 13]","[Pasir Panjang, Hong Leong Garden, Clementi ..."


In [375]:
# Columns Postal Sector and General Location apply explode function to convert each item in the list to a row
districts = districts.explode('Postal Sector')
districts = districts.explode('General Location')

# remove whitespace in postal sector
districts['Postal Sector'] = districts['Postal Sector'].str.strip()




In [376]:
districts

,Postal District,Postal Sector,General Location
0,1,01,Raffles Place
0,1,01,Cecil
0,1,01,Marina
0,1,01,People’s Park
0,1,02,Raffles Place
...,...,...,...
26,27,75,Sembawang
26,27,76,Yishun
26,27,76,Sembawang
27,28,79,Seletar


Resale flats

# read resale_flats.csv


In [377]:
resale_flats = pd.read_csv('../data/resale_flats.csv')

C:\Users\valen\AppData\Local\Temp\ipykernel_10684\671743954.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  resale_flats = pd.read_csv('../data/resale_flats.csv')


In [378]:
resale_flats

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,232000.0,2017-01,61 years 04 months,1979,10 TO 12,1,406
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,250000.0,2017-01,60 years 07 months,1978,01 TO 03,2,108
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,262000.0,2017-01,62 years 05 months,1980,01 TO 03,3,602
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,265000.0,2017-01,62 years 01 month,1980,04 TO 06,4,465
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,265000.0,2017-01,62 years 05 months,1980,01 TO 03,5,601
...,...,...,...,...,...,...,...,...,...,...,...,...
893732,YISHUN,EXECUTIVE,APARTMENT,142.0,YISHUN ST 61,456000.0,1999-12,NaN,1987,10 TO 12,287192,611
893733,YISHUN,EXECUTIVE,APARTMENT,142.0,YISHUN CTRL,408000.0,1999-12,NaN,1988,01 TO 03,287193,324
893734,YISHUN,EXECUTIVE,MAISONETTE,146.0,YISHUN AVE 6,469000.0,1999-12,NaN,1988,07 TO 09,287194,392
893735,YISHUN,EXECUTIVE,MAISONETTE,146.0,YISHUN RING RD,440000.0,1999-12,NaN,1988,04 TO 06,287195,356


In [379]:
# split month column which is in the format of YYYY-MM into 2 columns
resale_flats['month'] = resale_flats['month'].str.split('-')

resale_flats['year'] = resale_flats['month'].apply(lambda x: int(x[0]))
resale_flats['month'] = resale_flats['month'].apply(lambda x: int(x[1]))



In [380]:
# take only most recent 5 years of data
# sort indescending order of year then month

resale_flats = resale_flats[resale_flats['year'] >= 2017]
resale_flats = resale_flats.sort_values(by=['year', 'month'], ascending=False)
print(resale_flats.shape)
# take first 100 
resale_flats = resale_flats.head(1000)

(147534, 13)


In [381]:
# use openmap api to get postal code and district from the street name
# add dynamic programming to cache the results if same address is called again
dict1 = {}

count = 0
def get_info_from_street_name(address):
    global count
    count += 1
    if count % 1000 == 0:
        print(count)
    # if address is in dict1, return the value
    if address in dict1:
        return dict1[address]
    # else call the api and add to dict1
    else:
        
        url = "https://developers.onemap.sg/commonapi/search?searchVal={}&returnGeom=Y&getAddrDetails=Y".format(address)
        response = requests.get(url)
        result = response.json()['results'][0]
        postal = result['POSTAL']
        x = result['X']
        y = result['Y']
        lat = result['LATITUDE']
        lon = result['LONGITUDE']
        dict1[address] = (postal, x, y, lat, lon)
        
        return (postal, x, y, lat, lon)


In [382]:
# Apply get_info_from_street_name function to each row using a new column created by concat block and street name
resale_flats['street_name_with_block'] = resale_flats['block'] + ' ' + resale_flats['street_name']
resale_flats['postal'], resale_flats['x'], resale_flats['y'], resale_flats['lat'], resale_flats['lon'] = zip(*resale_flats['street_name_with_block'].apply(get_info_from_street_name))


1000


In [383]:
resale_flats

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block,year,street_name_with_block,postal,x,y,lat,lon
145971,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 3,290000.0,2,54 years,1978,07 TO 09,145972,314,2023,314 ANG MO KIO AVE 3,560314,29865.9980458226,38695.9702712912,1.36622707120636,103.850085858983
145972,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 8,331888.0,2,56 years 04 months,1980,07 TO 09,145973,510,2023,510 ANG MO KIO AVE 8,560510,29753.3009291003,39489.2179400064,1.37340092645025,103.849073244454
145973,ANG MO KIO,3 ROOM,Model A,70.0,ANG MO KIO AVE 1,530000.0,2,88 years 08 months,2012,01 TO 03,145974,308A,2023,308A ANG MO KIO AVE 1,561308,29198.1455183588,38613.7381223254,1.36548342757106,103.844084739929
145974,ANG MO KIO,3 ROOM,New Generation,82.0,ANG MO KIO AVE 1,418000.0,2,54 years,1978,04 TO 06,145975,223,2023,223 ANG MO KIO AVE 1,560223,28534.6432265872,38676.1728609148,1.36604808445916,103.838122716883
145975,ANG MO KIO,3 ROOM,Model A,70.0,ANG MO KIO AVE 1,630000.0,2,88 years 08 months,2012,16 TO 18,145976,308A,2023,308A ANG MO KIO AVE 1,561308,29198.1455183588,38613.7381223254,1.36548342757106,103.844084739929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146966,SEMBAWANG,4 ROOM,Model A,94.0,ADMIRALTY LINK,505000.0,2,86 years 06 months,2010,07 TO 09,146967,488,2023,488 ADMIRALTY LINK,750488,26123.124558188,48520.6620979905,1.45507803962368,103.81645291747
146967,SEMBAWANG,4 ROOM,Model A,93.0,CANBERRA CRES,609000.0,2,94 years 01 month,2018,10 TO 12,146968,120D,2023,120D CANBERRA CRES,754120,27886.0029991083,47601.6699954923,1.44676707236829,103.83229420147
146968,SEMBAWANG,4 ROOM,Model A,93.0,CANBERRA CRES,600000.0,2,94 years 09 months,2018,07 TO 09,146969,130A,2023,130A CANBERRA CRES,751130,27838.29959739,47380.1606274899,1.44476382182316,103.831865540071
146969,SEMBAWANG,4 ROOM,Model A,93.0,CANBERRA ST,588000.0,2,94 years 07 months,2018,07 TO 09,146970,107A,2023,107A CANBERRA ST,751107,27962.2371536177,47948.6182117146,1.44990474622,103.832979240964


In [398]:
# function to get district from postal code from districts table
def get_district_from_postal(postal):
    postal_sector = str(postal)[:2]
    value = districts[districts['Postal Sector'] == postal_sector]['Postal District'].values
    if not len(value):
        print(postal)
        return 'NIL'
    return value[0]



In [399]:
resale_flats['postal']

145971    560314
145972    560510
145973    561308
145974    560223
145975    561308
           ...  
146966    750488
146967    754120
146968    751130
146969    751107
146970    752128
Name: postal, Length: 1000, dtype: object

In [400]:
# apply get_district_from_postal function to each row using a new column created by postal code
resale_flats['district'] = resale_flats['postal'].apply(get_district_from_postal)


NIL


In [401]:
resale_flats

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block,year,street_name_with_block,postal,x,y,lat,lon,district
145971,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 3,290000.0,2,54 years,1978,07 TO 09,145972,314,2023,314 ANG MO KIO AVE 3,560314,29865.9980458226,38695.9702712912,1.36622707120636,103.850085858983,20
145972,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 8,331888.0,2,56 years 04 months,1980,07 TO 09,145973,510,2023,510 ANG MO KIO AVE 8,560510,29753.3009291003,39489.2179400064,1.37340092645025,103.849073244454,20
145973,ANG MO KIO,3 ROOM,Model A,70.0,ANG MO KIO AVE 1,530000.0,2,88 years 08 months,2012,01 TO 03,145974,308A,2023,308A ANG MO KIO AVE 1,561308,29198.1455183588,38613.7381223254,1.36548342757106,103.844084739929,20
145974,ANG MO KIO,3 ROOM,New Generation,82.0,ANG MO KIO AVE 1,418000.0,2,54 years,1978,04 TO 06,145975,223,2023,223 ANG MO KIO AVE 1,560223,28534.6432265872,38676.1728609148,1.36604808445916,103.838122716883,20
145975,ANG MO KIO,3 ROOM,Model A,70.0,ANG MO KIO AVE 1,630000.0,2,88 years 08 months,2012,16 TO 18,145976,308A,2023,308A ANG MO KIO AVE 1,561308,29198.1455183588,38613.7381223254,1.36548342757106,103.844084739929,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146966,SEMBAWANG,4 ROOM,Model A,94.0,ADMIRALTY LINK,505000.0,2,86 years 06 months,2010,07 TO 09,146967,488,2023,488 ADMIRALTY LINK,750488,26123.124558188,48520.6620979905,1.45507803962368,103.81645291747,27
146967,SEMBAWANG,4 ROOM,Model A,93.0,CANBERRA CRES,609000.0,2,94 years 01 month,2018,10 TO 12,146968,120D,2023,120D CANBERRA CRES,754120,27886.0029991083,47601.6699954923,1.44676707236829,103.83229420147,27
146968,SEMBAWANG,4 ROOM,Model A,93.0,CANBERRA CRES,600000.0,2,94 years 09 months,2018,07 TO 09,146969,130A,2023,130A CANBERRA CRES,751130,27838.29959739,47380.1606274899,1.44476382182316,103.831865540071,27
146969,SEMBAWANG,4 ROOM,Model A,93.0,CANBERRA ST,588000.0,2,94 years 07 months,2018,07 TO 09,146970,107A,2023,107A CANBERRA ST,751107,27962.2371536177,47948.6182117146,1.44990474622,103.832979240964,27


In [402]:
# read the private_transactions csv file
private_transactions = pd.read_csv('../data/private_transaction.csv')
# and private_rental
private_rental = pd.read_csv('../data/private_rental.csv')


In [403]:
private_transactions.head()

,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,nettPrice,street,project,marketSegment
0,524.3,-,1,220,3,5500000,Semi-detached,5,Land,Freehold,NaN,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR
1,308.0,-,1,918,3,5000000,Semi-detached,5,Land,Freehold,NaN,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR
2,314.0,-,1,618,3,4750000,Semi-detached,5,Land,Freehold,NaN,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR
3,308.0,-,1,921,3,5200000,Semi-detached,5,Land,Freehold,NaN,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR
4,159.3,-,1,320,3,2630000,Terrace,5,Land,Freehold,NaN,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR


In [404]:
def private_get_month_year(date):
    date = str(date)
    # date is in format of MYY or MMYY
    # if date is in format of MYY, add 0 in front
    if len(date) == 3:
        date = '0' + date
    month = date[:2]
    year = date[2:]
    # add prefix to year 20
    year = '20' + year
    return (month, year)

In [405]:
private_get_month_year(618)

('06', '2018')

In [406]:
private_rental

,areaSqm,leaseDate,propertyType,district,areaSqft,noOfBedRoom,rent,street,project
0,100-110,318,Non-landed Properties,23,1100-1200,2.0,2300,HILLVIEW AVENUE,MERAWOODS
1,120-130,318,Non-landed Properties,23,1300-1400,3.0,2300,HILLVIEW AVENUE,MERAWOODS
2,90-100,218,Non-landed Properties,23,1000-1100,2.0,2100,HILLVIEW AVENUE,MERAWOODS
3,90-100,218,Non-landed Properties,23,1000-1100,2.0,2150,HILLVIEW AVENUE,MERAWOODS
4,190-200,318,Non-landed Properties,15,2100-2200,3.0,4700,ELLIOT ROAD,ELLIOT AT THE EAST COAST
...,...,...,...,...,...,...,...,...,...
477802,80-90,123,Executive Condominium,19,800-900,2.0,2950,PUNGGOL FIELD,PRIVE
477803,90-100,123,Executive Condominium,19,1000-1100,3.0,4300,PUNGGOL FIELD,PRIVE
477804,100-110,123,Executive Condominium,19,1000-1100,2.0,4500,PUNGGOL FIELD,PRIVE
477805,100-110,123,Executive Condominium,19,1000-1100,3.0,4300,PUNGGOL FIELD,PRIVE


In [407]:
# apply private_get_month_year function to each row, creating 2 new columns for month and year
private_transactions['month'], private_transactions['year'] = zip(*private_transactions['contractDate'].apply(private_get_month_year))
# same for private_rental on leaseDate
private_rental['month'], private_rental['year'] = zip(*private_rental['leaseDate'].apply(private_get_month_year))

In [408]:
private_transactions

,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,nettPrice,street,project,marketSegment,month,year
0,524.3,-,1,220,3,5500000,Semi-detached,5,Land,Freehold,NaN,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,02,2020
1,308.0,-,1,918,3,5000000,Semi-detached,5,Land,Freehold,NaN,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,09,2018
2,314.0,-,1,618,3,4750000,Semi-detached,5,Land,Freehold,NaN,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,06,2018
3,308.0,-,1,921,3,5200000,Semi-detached,5,Land,Freehold,NaN,ZEHNDER ROAD,LANDED HOUSING DEVELOPMENT,RCR,09,2021
4,159.3,-,1,320,3,2630000,Terrace,5,Land,Freehold,NaN,NEO PEE TECK LANE,LANDED HOUSING DEVELOPMENT,RCR,03,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130978,304.1,-,1,422,3,3430000,Terrace,27,Land,Freehold,NaN,GOODLINK PARK,GOODLINK PARK,OCR,04,2022
130979,901.9,-,1,522,3,7300000,Detached,28,Land,999 yrs lease commencing from 1879,NaN,GERALD CRESCENT,GERALD MUGLISTON ESTATE,OCR,05,2022
130980,307.9,-,1,722,3,4250000,Semi-detached,28,Land,Freehold,NaN,GERALD CRESCENT,GERALD MUGLISTON ESTATE,OCR,07,2022
130981,684.6,-,1,522,3,6000000,Detached,28,Land,999 yrs lease commencing from 1879,NaN,GERALD CRESCENT,GERALD MUGLISTON ESTATE,OCR,05,2022


In [409]:
private_rental

,areaSqm,leaseDate,propertyType,district,areaSqft,noOfBedRoom,rent,street,project,month,year
0,100-110,318,Non-landed Properties,23,1100-1200,2.0,2300,HILLVIEW AVENUE,MERAWOODS,03,2018
1,120-130,318,Non-landed Properties,23,1300-1400,3.0,2300,HILLVIEW AVENUE,MERAWOODS,03,2018
2,90-100,218,Non-landed Properties,23,1000-1100,2.0,2100,HILLVIEW AVENUE,MERAWOODS,02,2018
3,90-100,218,Non-landed Properties,23,1000-1100,2.0,2150,HILLVIEW AVENUE,MERAWOODS,02,2018
4,190-200,318,Non-landed Properties,15,2100-2200,3.0,4700,ELLIOT ROAD,ELLIOT AT THE EAST COAST,03,2018
...,...,...,...,...,...,...,...,...,...,...,...
477802,80-90,123,Executive Condominium,19,800-900,2.0,2950,PUNGGOL FIELD,PRIVE,01,2023
477803,90-100,123,Executive Condominium,19,1000-1100,3.0,4300,PUNGGOL FIELD,PRIVE,01,2023
477804,100-110,123,Executive Condominium,19,1000-1100,2.0,4500,PUNGGOL FIELD,PRIVE,01,2023
477805,100-110,123,Executive Condominium,19,1000-1100,3.0,4300,PUNGGOL FIELD,PRIVE,01,2023


Save all transformed files before load



In [410]:
# save all 3 dataframes 
resale_flats.to_csv('../data/resale_flats_transformed.csv', index=False)
private_transactions.to_csv('../data/private_transactions_transformed.csv', index=False)
private_rental.to_csv('../data/private_rental_transformed.csv', index=False)
